# Library Import

In [ ]:
!pip install gdown

In [ ]:
import pickle
import gzip
import numpy as np
import gdown
import zipfile
import os
from PIL import Image
from sklearn.metrics import confusion_matrix

# Load Data

## Load MNIST Data

In [ ]:
url = 'https://drive.google.com/uc?id=1bf3Fb_5VelMq8kT0q-Zc8sRpxvYjJ05S'
output = 'mnist.pkl.gz'

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bf3Fb_5VelMq8kT0q-Zc8sRpxvYjJ05S
To: /content/mnist.pkl.gz
100%|██████████| 17.1M/17.1M [00:00<00:00, 67.3MB/s]


'mnist.pkl.gz'

In [ ]:
# Opening and decompressing .gzip file
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')

# Unpickling .pkl file saved as byte stream on disk to python objects
trainingData, validationData, testingData = pickle.load(f, encoding='latin1')
f.close()


In [ ]:
# Training feature vectors and training target extracted from trainingData (target represented using one-hot format)
trainingTarget = np.array([1 if  trainingData[1][i] == t else 0  for i in range(len(trainingData[1])) for t in range(10)]).reshape(-1,10)
trainingData = trainingData[0]

# Validation feature vectors and validation target extracted from validationData
validationTarget = validationData[1]
validationData = validationData[0]

# Testing feature vectors and testing target extracted from testingData
testingTarget = testingData[1]
testingData = testingData[0]

## Load Data USPS

In [ ]:
url = 'https://drive.google.com/uc?id=11wHV2TrUAo9Zk5OgkX1Zv1XJuxC-IhmD'
output = 'USPSdata.zip'

# install file ZIP
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=11wHV2TrUAo9Zk5OgkX1Zv1XJuxC-IhmD
From (redirected): https://drive.google.com/uc?id=11wHV2TrUAo9Zk5OgkX1Zv1XJuxC-IhmD&confirm=t&uuid=34bf34d3-57a0-4447-8812-b2cc5033edc1
To: /content/USPSdata.zip
100%|██████████| 33.0M/33.0M [00:01<00:00, 20.1MB/s]


'USPSdata.zip'

In [ ]:
# extract file ZIP
with zipfile.ZipFile('USPSdata.zip', 'r') as zip_ref:
    zip_ref.extractall('USPSdata')

In [ ]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/USPSdata/Numerals'
savedImg = []

In [ ]:

for j in range(0, 10):
    curFolderPath = curPath + '/' + str(j)
    imgs = os.listdir(curFolderPath)

    for img in imgs:
        curImg = curFolderPath + '/' + img

        if curImg[-3:] == 'png':
            # Open image resize it, and save image array as well as labels in 2 lists
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

# Convolutional Neural Network

In [ ]:
import keras
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten

In [ ]:
#CNN
num_classes = len(trainingTarget[0])
CNNclassifier = Sequential()
CNNclassifier.add(Conv2D(20, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
CNNclassifier.add(MaxPooling2D(pool_size=(2, 2)))
CNNclassifier.add(Conv2D(20, (3, 3), activation='relu'))
CNNclassifier.add(MaxPooling2D(pool_size=(2, 2)))
CNNclassifier.add(Flatten())
CNNclassifier.add(Dense(units=200, activation='relu'))
CNNclassifier.add(Dropout(0.2))
CNNclassifier.add(Dense(units=200, activation='relu'))
CNNclassifier.add(Dropout(0.2))
CNNclassifier.add(Dense(units=num_classes, activation='softmax'))
CNNclassifier.compile(optimizer='Adamax', loss='categorical_crossentropy',metrics=['accuracy'])
history = CNNclassifier.fit(trainingData.reshape(-1, 28, 28, 1), trainingTarget, batch_size=128, epochs=10,
verbose=True,validation_data=(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 28s 66ms/step - accuracy: 0.6994 - loss: 0.9640 - val_accuracy: 0.9618 - val_loss: 0.1362
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 66ms/step - accuracy: 0.9512 - loss: 0.1661 - val_accuracy: 0.9741 - val_loss: 0.0876
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.9644 - loss: 0.1159 - val_accuracy: 0.9789 - val_loss: 0.0717
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 45s 74ms/step - accuracy: 0.9726 - loss: 0.0894 - val_accuracy: 0.9814 - val_loss: 0.0626
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 65ms/step - accuracy: 0.9777 - loss: 0.0735 - val_accuracy: 0.9830 - val_loss: 0.0562
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 63ms/step - accuracy: 0.9804 - loss: 0.0628 - val_accuracy: 0.9838 - val_loss: 0.0521
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 65ms/step - accuracy: 0.9819 - loss: 0.0584 - val_accuracy: 0.9851 - val_loss: 0.0510
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9855 - loss: 0.0481 - 

In [ ]:
loss,valAccuracy = CNNclassifier.evaluate(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes), verbose=False)
loss,testAccuracy = CNNclassifier.evaluate(testingData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(testingTarget, num_classes), verbose=False)
loss,testUSPSAccuracy = CNNclassifier.evaluate(np.array(USPSMat).reshape(-1, 28, 28, 1), keras.utils.to_categorical(USPSTar, num_classes), verbose=False)
cnnMNISTPrediction = np.argmax(CNNclassifier.predict(testingData.reshape(-1, 28, 28, 1)), axis=1)
cnnUSPSPrediction = np.argmax(CNNclassifier.predict(np.array(USPSMat).reshape(-1, 28, 28, 1)), axis=1)

print ('---------------CNN----------------\n')
print('Validation accuracy    = ', valAccuracy*100)
print('MNIST Testing accuracy = ', testAccuracy*100)
print('USPS Testing accuracy  = ', testUSPSAccuracy*100,"\n")
print("\nMNIST Confusion Matrix: \n\n",confusion_matrix(testingTarget, cnnMNISTPrediction))
print("\nUSPS Confusion Matrix: \n\n",confusion_matrix(USPSTar, cnnUSPSPrediction))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step
---------------CNN----------------

Validation accuracy    =  98.79000186920166
MNIST Testing accuracy =  98.9300012588501
USPS Testing accuracy  =  63.198161125183105 


MNIST Confusion Matrix: 

 [[ 975    0    0    0    1    0    1    1    2    0]
 [   0 1131    3    0    0    0    1    0    0    0]
 [   2    0 1020    1    1    0    0    7    1    0]
 [   1    0    1  999    0    3    0    3    2    1]
 [   0    0    1    0  975    0    1    0    1    4]
 [   2    0    1    5    0  880    2    1    1    0]
 [   5    2    0    1    2    4  942    0    2    0]
 [   0    0    4    2    0    0    0 1017    1    4]
 [   3    0    2    2    0    2    0    3  956    6]
 [   3    2    0    1    4    1    0    0    0  998]]

USPS Confusion Matrix: 

 [[ 696    0   65   24  400    8   10   32   29  736]
 [  82  732  169   52  350   23   11  492   69   20]
 [  39    2 1667   85   16   95   15   16   57    7]
 [

# DenseNet

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dense, Flatten, Dropout, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax

In [ ]:
def densenet_block(x, growth_rate, layers):
    for _ in range(layers):
        bn = BatchNormalization()(x)
        relu = Activation('relu')(bn)

        bottleneck = Conv2D(4 * growth_rate, (1, 1), padding='same', activation='relu')(relu)

        conv = Conv2D(growth_rate, (3, 3), padding='same')(bottleneck)

        x = Concatenate()([x, conv])
    return x

def transition_layer(x, compression_factor=0.5):
    bn = BatchNormalization()(x)
    relu = Activation('relu')(bn)

    conv = Conv2D(int(x.shape[-1] * compression_factor), (1, 1), padding='same', activation='relu')(relu)

    x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv)
    return x


def create_densenet_model(input_shape, num_classes, growth_rate=32, num_dense_blocks=3, layers_per_block=4, compression_factor=0.5):
    inputs = Input(shape=input_shape)

    # Initial Conv layer
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Add DenseNet blocks with transition layers
    for i in range(num_dense_blocks):
        x = densenet_block(x, growth_rate, layers_per_block)

        if i != num_dense_blocks - 1:
            x = transition_layer(x, compression_factor)

    x = GlobalAveragePooling2D()(x)

    # Fully connected layers
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)


    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    model.compile(optimizer=Adamax(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
DenseNetModel = create_densenet_model((28, 28, 1), num_classes)

history = DenseNetModel.fit(
    trainingData.reshape(-1, 28, 28, 1),
    trainingTarget,
    batch_size=128,
    epochs=10,
    verbose=True,
    validation_data=(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes))
)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 700s 2s/step - accuracy: 0.7529 - loss: 0.7096 - val_accuracy: 0.2347 - val_loss: 2.1990
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 737s 2s/step - accuracy: 0.9833 - loss: 0.0570 - val_accuracy: 0.9509 - val_loss: 0.1660
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 683s 2s/step - accuracy: 0.9875 - loss: 0.0403 - val_accuracy: 0.9855 - val_loss: 0.0530
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 684s 2s/step - accuracy: 0.9926 - loss: 0.0255 - val_accuracy: 0.9783 - val_loss: 0.0852
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 753s 2s/step - accuracy: 0.9934 - loss: 0.0221 - val_accuracy: 0.9507 - val_loss: 0.2140
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 729s 2s/step - accuracy: 0.9941 - loss: 0.0184 - val_accuracy: 0.9839 - val_loss: 0.0685
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 688s 2s/step - accuracy: 0.9951 - loss: 0.0144 - val_accuracy: 0.9902 - val_loss: 0.0366
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 751s 2s/step - accuracy: 0.9955 - loss: 0.0137 - val_accu

In [ ]:
from sklearn.metrics import confusion_matrix

loss, valAccuracy = DenseNetModel.evaluate(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes), verbose=False)
loss, testAccuracy = DenseNetModel.evaluate(testingData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(testingTarget, num_classes), verbose=False)
loss, testUSPSAccuracy = DenseNetModel.evaluate(np.array(USPSMat).reshape(-1, 28, 28, 1), keras.utils.to_categorical(USPSTar, num_classes), verbose=False)
densenetMNISTPrediction = np.argmax(DenseNetModel.predict(testingData.reshape(-1, 28, 28, 1)), axis=1)
densenetUSPSPrediction = np.argmax(DenseNetModel.predict(np.array(USPSMat).reshape(-1, 28, 28, 1)), axis=1)

print('---------------DenseNet----------------\n')
print('Validation accuracy    = ', valAccuracy * 100)
print('MNIST Testing accuracy = ', testAccuracy * 100)
print('USPS Testing accuracy  = ', testUSPSAccuracy * 100, "\n")
print("\nMNIST Confusion Matrix: \n\n", confusion_matrix(testingTarget, densenetMNISTPrediction))
print("\nUSPS Confusion Matrix: \n\n", confusion_matrix(USPSTar, densenetUSPSPrediction))


313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 105ms/step
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step
---------------DenseNet----------------

Validation accuracy    =  98.86000156402588
MNIST Testing accuracy =  99.1100013256073
USPS Testing accuracy  =  76.55882835388184 


MNIST Confusion Matrix: 

 [[ 974    0    1    0    2    0    2    1    0    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   0    0 1028    0    3    0    0    1    0    0]
 [   0    1    0 1000    0    8    0    0    0    1]
 [   0    0    0    0  982    0    0    0    0    0]
 [   1    0    0    2    0  888    1    0    0    0]
 [   1    3    0    0    9    1  944    0    0    0]
 [   0    5    8    0    1    0    0 1007    0    7]
 [   1    0    1    0    3    1    0    0  965    3]
 [   0    0    0    0   20    0    0    0    1  988]]

USPS Confusion Matrix: 

 [[ 927    0  547    0  273   12   65    0  147   29]
 [   2 1118  163   21  508   18  137    6   25    2]
 [   0    2 1955    8    5    5    0    2   21

# Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1
)

## CNN

In [ ]:
history = CNNclassifier.fit(datagen.flow(trainingData.reshape(-1, 28, 28, 1), trainingTarget, batch_size=128),
                    epochs=10, validation_data=(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes)))

loss, valAccuracy = CNNclassifier.evaluate(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes), verbose=False)
loss, testAccuracy = CNNclassifier.evaluate(testingData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(testingTarget, num_classes), verbose=False)
loss, testUSPSAccuracy = CNNclassifier.evaluate(np.array(USPSMat).reshape(-1, 28, 28, 1), keras.utils.to_categorical(USPSTar, num_classes), verbose=False)

cnnMNISTPrediction = np.argmax(CNNclassifier.predict(testingData.reshape(-1, 28, 28, 1)), axis=1)
cnnUSPSPrediction = np.argmax(CNNclassifier.predict(np.array(USPSMat).reshape(-1, 28, 28, 1)), axis=1)

print ('---------------CNN----------------\n')
print('Validation accuracy    = ', valAccuracy*100)
print('MNIST Testing accuracy = ', testAccuracy*100)
print('USPS Testing accuracy  = ', testUSPSAccuracy*100,"\n")
print("\nMNIST Confusion Matrix: \n\n", confusion_matrix(testingTarget, cnnMNISTPrediction))
print("\nUSPS Confusion Matrix: \n\n", confusion_matrix(USPSTar, cnnUSPSPrediction))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.9039 - loss: 0.3153 - val_accuracy: 0.9842 - val_loss: 0.0494
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.9456 - loss: 0.1718 - val_accuracy: 0.9871 - val_loss: 0.0434
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 105ms/step - accuracy: 0.9557 - loss: 0.1465 - val_accuracy: 0.9866 - val_loss: 0.0444
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.9616 - loss: 0.1255 - val_accuracy: 0.9865 - val_loss: 0.0431
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.9637 - loss: 0.1194 - val_accuracy: 0.9887 - val_loss: 0.0393
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 106ms/step - accuracy: 0.9657 - loss: 0.1090 - val_accuracy: 0.9899 - val_loss: 0.0376
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 107ms/step - accuracy: 0.9711 - loss: 0.0949 - val_accuracy: 0.9899 - val_loss: 0.0358
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 107ms/step - accuracy: 0.9717 - loss: 0.0928 - val

## DenseNet

In [ ]:
history = DenseNetModel.fit(datagen.flow(trainingData.reshape(-1, 28, 28, 1), trainingTarget, batch_size=128),
                    epochs=10, validation_data=(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes)))

# Evaluasi model
loss, valAccuracy = DenseNetModel.evaluate(validationData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(validationTarget, num_classes), verbose=False)
loss, testAccuracy = DenseNetModel.evaluate(testingData.reshape(-1, 28, 28, 1), keras.utils.to_categorical(testingTarget, num_classes), verbose=False)
loss, testUSPSAccuracy = DenseNetModel.evaluate(np.array(USPSMat).reshape(-1, 28, 28, 1), keras.utils.to_categorical(USPSTar, num_classes), verbose=False)

# Prediksi
densenetMNISTPrediction = np.argmax(DenseNetModel.predict(testingData.reshape(-1, 28, 28, 1)), axis=1)
densenetUSPSPrediction = np.argmax(DenseNetModel.predict(np.array(USPSMat).reshape(-1, 28, 28, 1)), axis=1)

# Output hasil
print ('---------------DenseNet----------------\n')
print('Validation accuracy    = ', valAccuracy*100)
print('MNIST Testing accuracy = ', testAccuracy*100)
print('USPS Testing accuracy  = ', testUSPSAccuracy*100,"\n")
print("\nMNIST Confusion Matrix: \n\n", confusion_matrix(testingTarget, densenetMNISTPrediction))
print("\nUSPS Confusion Matrix: \n\n", confusion_matrix(USPSTar, densenetUSPSPrediction))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


391/391 ━━━━━━━━━━━━━━━━━━━━ 705s 2s/step - accuracy: 0.9864 - loss: 0.0479 - val_accuracy: 0.9829 - val_loss: 0.0784
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 708s 2s/step - accuracy: 0.9904 - loss: 0.0311 - val_accuracy: 0.9910 - val_loss: 0.0366
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 711s 2s/step - accuracy: 0.9905 - loss: 0.0319 - val_accuracy: 0.9404 - val_loss: 0.2392
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 729s 2s/step - accuracy: 0.9919 - loss: 0.0274 - val_accuracy: 0.9604 - val_loss: 0.1536
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 744s 2s/step - accuracy: 0.9928 - loss: 0.0241 - val_accuracy: 0.9914 - val_loss: 0.0334
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 756s 2s/step - accuracy: 0.9929 - loss: 0.0237 - val_accuracy: 0.9922 - val_loss: 0.0340
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.9935 - loss: 0.0219 - val_accuracy: 0.9914 - val_loss: 0.0365
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 738s 2s/step - accuracy: 0.9937 - loss: 0.0223 - val_accuracy: 0.979